In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import yfinance as yf

In [9]:
xmr = yf.Ticker("XMR-USD")
df_xmr = yf.download(tickers = "XMR-USD",
                     period = "max",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [19]:
df_xmr

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-11-09,112.531998,123.404999,112.219002,120.779999,120.779999,86864600
2017-11-10,121.344002,121.665001,101.757004,105.585999,105.585999,84614000
2017-11-11,105.750000,127.106003,103.877998,119.615997,119.615997,107708000
2017-11-12,119.597000,133.675003,110.617996,123.856003,123.856003,144948000
2017-11-13,128.960007,136.528000,120.921997,123.402000,123.402000,116200000
...,...,...,...,...,...,...
2023-10-09,156.190552,156.192932,151.908813,153.795502,153.795502,61203300
2023-10-10,153.790405,154.468155,152.097107,152.215652,152.215652,54504811
2023-10-11,152.220093,153.929443,150.970428,153.842102,153.842102,61162383


In [21]:
df_xmr.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2165 entries, 2017-11-09 to 2023-10-13
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2165 non-null   float64
 1   High       2165 non-null   float64
 2   Low        2165 non-null   float64
 3   Close      2165 non-null   float64
 4   Adj Close  2165 non-null   float64
 5   Volume     2165 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 118.4 KB
